## Overview Technical BDA 50k

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# Import numpy, pandas, matpltlib.pyplot, sklearn modules and seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

# Import KNeighborsClassifier from sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier

# Import DecisionTreeClassifier from sklearn.tree
from sklearn.tree import DecisionTreeClassifier

# Import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [3]:
# File location and type
file_location = "/FileStore/tables/US_Accidents_Dec19_50.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_spark = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_spark)

ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
A-1,MapQuest,201,3,08-02-16 5:46,08-02-16 11:00,39.865147,-84.058723,null,null,0.01,Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.,null,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,08-02-16 5:58,36.9,null,91,29.68,10,Calm,null,0.02,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Night,Night,Night
A-2,MapQuest,201,2,08-02-16 6:07,08-02-16 6:37,39.928059,-82.831184,null,null,0.01,Accident on Brice Rd at Tussing Rd. Expect delays.,2584,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,08-02-16 5:51,37.9,null,100,29.65,10,Calm,null,0,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Night,Night,Day
A-3,MapQuest,201,2,08-02-16 6:49,08-02-16 7:19,39.063148,-84.032608,null,null,0.01,Accident on OH-32 State Route 32 Westbound at Dela Palma Rd. Expect delays.,null,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,08-02-16 6:56,36,33.3,100,29.67,10,SW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,Night,Night,Day,Day
A-4,MapQuest,201,3,08-02-16 7:23,08-02-16 7:53,39.747753,-84.205582,null,null,0.01,Accident on I-75 Southbound at Exits 52 52B US-35. Expect delays.,null,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,08-02-16 7:38,35.1,31,96,29.64,9,SW,4.6,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Day,Day,Day
A-5,MapQuest,201,2,08-02-16 7:39,08-02-16 8:09,39.627781,-84.188354,null,null,0.01,Accident on McEwen Rd at OH-725 Miamisburg Centerville Rd. Expect delays.,null,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,08-02-16 7:53,36,33.3,89,29.65,6,SW,3.5,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,Day,Day,Day,Day
A-6,MapQuest,201,3,08-02-16 7:44,08-02-16 8:14,40.10059,-82.925194,null,null,0.01,Accident on I-270 Outerbelt Northbound near Exit 29 OH-3 State St. Expect delays.,null,Westerville Rd,R,Westerville,Franklin,OH,43081,US,US/Eastern,KCMH,08-02-16 7:51,37.9,35.5,97,29.63,7,SSW,3.5,0.03,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-7,MapQuest,201,2,08-02-16 7:59,08-02-16 8:29,39.758274,-84.230507,null,null,0,Accident on Oakridge Dr at Woodward Ave. Expect delays.,376,N Woodward Ave,R,Dayton,Montgomery,OH,45417-2476,US,US/Eastern,KDAY,08-02-16 7:56,34,31,100,29.66,7,WSW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-8,MapQuest,201,3,08-02-16 7:59,08-02-16 8:29,39.770382,-84.194901,null,null,0.01,Accident on I-75 Southbound at Exit 54B Grand Ave. Expect delays.,null,N Main St,R,Dayton,Montgomery,OH,45405,US,US/Eastern,KDAY,08-02-16 7:56,34,31,100,29.66,7,WSW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-9,MapQuest,201,2,08-02-16 8:00,08-02-16 8:30,39.778061,-84.172005,null,null,0,Accident on Notre Dame Ave at Warner Ave. Expect delays.,99,Notre Dame Ave,L,Dayton,Montgomery,OH,45404-1923,US,US/Eastern,KFFO,08-02-16 7:58,33.3,null,99,29.67,5,SW,1.2,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-10,MapQuest,201,3,08-02-16 8:10,08-02-16 8:40,40.10059,-82.925194,null,null,0.01,Right hand shoulder blocked 

In [4]:
# Create a view or table

temp_table_name = "US_Accidents_Dec19_50_csv"

df_spark.createOrReplaceTempView(temp_table_name)

In [5]:
%sql

/* Query the created temp table in a SQL cell */

select * from `US_Accidents_Dec19_50_csv`

ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
A-1,MapQuest,201,3,08-02-16 5:46,08-02-16 11:00,39.865147,-84.058723,null,null,0.01,Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4.,null,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,08-02-16 5:58,36.9,null,91,29.68,10,Calm,null,0.02,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Night,Night,Night
A-2,MapQuest,201,2,08-02-16 6:07,08-02-16 6:37,39.928059,-82.831184,null,null,0.01,Accident on Brice Rd at Tussing Rd. Expect delays.,2584,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,08-02-16 5:51,37.9,null,100,29.65,10,Calm,null,0,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Night,Night,Day
A-3,MapQuest,201,2,08-02-16 6:49,08-02-16 7:19,39.063148,-84.032608,null,null,0.01,Accident on OH-32 State Route 32 Westbound at Dela Palma Rd. Expect delays.,null,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,08-02-16 6:56,36,33.3,100,29.67,10,SW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,Night,Night,Day,Day
A-4,MapQuest,201,3,08-02-16 7:23,08-02-16 7:53,39.747753,-84.205582,null,null,0.01,Accident on I-75 Southbound at Exits 52 52B US-35. Expect delays.,null,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,08-02-16 7:38,35.1,31,96,29.64,9,SW,4.6,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Night,Day,Day,Day
A-5,MapQuest,201,2,08-02-16 7:39,08-02-16 8:09,39.627781,-84.188354,null,null,0.01,Accident on McEwen Rd at OH-725 Miamisburg Centerville Rd. Expect delays.,null,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,08-02-16 7:53,36,33.3,89,29.65,6,SW,3.5,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,Day,Day,Day,Day
A-6,MapQuest,201,3,08-02-16 7:44,08-02-16 8:14,40.10059,-82.925194,null,null,0.01,Accident on I-270 Outerbelt Northbound near Exit 29 OH-3 State St. Expect delays.,null,Westerville Rd,R,Westerville,Franklin,OH,43081,US,US/Eastern,KCMH,08-02-16 7:51,37.9,35.5,97,29.63,7,SSW,3.5,0.03,Light Rain,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-7,MapQuest,201,2,08-02-16 7:59,08-02-16 8:29,39.758274,-84.230507,null,null,0,Accident on Oakridge Dr at Woodward Ave. Expect delays.,376,N Woodward Ave,R,Dayton,Montgomery,OH,45417-2476,US,US/Eastern,KDAY,08-02-16 7:56,34,31,100,29.66,7,WSW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-8,MapQuest,201,3,08-02-16 7:59,08-02-16 8:29,39.770382,-84.194901,null,null,0.01,Accident on I-75 Southbound at Exit 54B Grand Ave. Expect delays.,null,N Main St,R,Dayton,Montgomery,OH,45405,US,US/Eastern,KDAY,08-02-16 7:56,34,31,100,29.66,7,WSW,3.5,null,Overcast,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-9,MapQuest,201,2,08-02-16 8:00,08-02-16 8:30,39.778061,-84.172005,null,null,0,Accident on Notre Dame Ave at Warner Ave. Expect delays.,99,Notre Dame Ave,L,Dayton,Montgomery,OH,45404-1923,US,US/Eastern,KFFO,08-02-16 7:58,33.3,null,99,29.67,5,SW,1.2,null,Mostly Cloudy,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,Day,Day,Day,Day
A-10,MapQuest,201,3,08-02-16 8:10,08-02-16 8:40,40.10059,-82.925194,null,null,0.01,Right hand shoulder blocked 

In [6]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "US_Accidents_Dec19_50_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [7]:
df = df_spark.toPandas()

In [8]:
# Convert Start_Time and End_Time to datetypes
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract year, month, day, hour and weekday
df['Year']=df['Start_Time'].dt.year
df['Month']=df['Start_Time'].dt.strftime('%b')
df['Day']=df['Start_Time'].dt.day
df['Hour']=df['Start_Time'].dt.hour
df['Weekday']=df['Start_Time'].dt.strftime('%a')

# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer
td='Time_Duration(min)'
df[td]=round((df['End_Time']-df['Start_Time'])/np.timedelta64(1,'m'))

In [9]:
df['TMC']=df['TMC'].astype(float)
df['Severity']=df['Severity'].astype(float)
df['Start_Lat']=df['Start_Lat'].astype(float)
df['Start_Lng']=df['Start_Lng'].astype(float)
df['End_Lat']=df['End_Lat'].astype(float)
df['End_Lng']=df['End_Lng'].astype(float)
df['Distance(mi)']=df['Distance(mi)'].astype(float)
df['Number']=df['Number'].astype(float)
df['Temperature(F)']=df['Temperature(F)'].astype(float)
df['Wind_Chill(F)']=df['Wind_Chill(F)'].astype(float)
df['Humidity(%)']=df['Humidity(%)'].astype(float)
df['Pressure(in)']=df['Pressure(in)'].astype(float)
df['Visibility(mi)']=df['Visibility(mi)'].astype(float)
df['Wind_Speed(mph)']=df['Wind_Speed(mph)'].astype(float)
df['Precipitation(in)']=df['Precipitation(in)'].astype(float)
df['Amenity']=df['Amenity'].astype(bool)
df['Bump']=df['Bump'].astype(bool)
df['Crossing']=df['Crossing'].astype(bool)
df['Give_Way']=df['Give_Way'].astype(bool)
df['Junction']=df['Junction'].astype(bool)
df['No_Exit']=df['No_Exit'].astype(bool)
df['Railway']=df['Railway'].astype(bool)
df['Roundabout']=df['Roundabout'].astype(bool)
df['Station']=df['Station'].astype(bool)
df['Stop']=df['Stop'].astype(bool)
df['Traffic_Calming']=df['Traffic_Calming'].astype(bool)
df['Traffic_Signal']=df['Traffic_Signal'].astype(bool)
df['Turning_Loop']=df['Turning_Loop'].astype(bool)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505420 entries, 0 to 505419
Data columns (total 55 columns):
ID 505420 non-null object
Source 505420 non-null object
TMC 505420 non-null float64
Severity 505420 non-null float64
Start_Time 505420 non-null datetime64[ns]
End_Time 505420 non-null datetime64[ns]
Start_Lat 505420 non-null float64
Start_Lng 505420 non-null float64
End_Lat 0 non-null float64
End_Lng 0 non-null float64
Distance(mi) 505420 non-null float64
Description 505420 non-null object
Number 178401 non-null float64
Street 505420 non-null object
Side 505420 non-null object
City 505398 non-null object
County 505420 non-null object
State 505420 non-null object
Zipcode 505358 non-null object
Country 505420 non-null object
Timezone 505358 non-null object
Airport_Code 505358 non-null object
Weather_Timestamp 501250 non-null object
Temperature(F) 498104 non-null float64
Wind_Chill(F) 62361 non-null float64
Humidity(%) 497437 non-null float64
Pressure(in) 499622 non-null float64
Visibility(mi) 494779 non-null float64
Wind_Direction 501218 non-null object
Wind_Speed(mph) 415679 non-null float64
Precipitation(in) 52674 non-null float64
Weather_Condition 495132 non-null object
Amenity 505420 non-null bool
Bump 505420 non-null bool
Crossing 505420 non-null bool
Give_Way 505420 non-null bool
Junction 505420 non-null bool
No_Exit 505420 non-null bool
Railway 505420 non-null bool
Roundabout 505420 non-null bool
Station 505420 non-null bool
Stop 505420 non-null bool
Traffic_Calming 505420 non-null bool
Traffic_Signal 505420 non-null bool
Turning_Loop 505420 non-null bool
Sunrise_Sunset 505398 non-null object
Civil_Twilight 505398 non-null object
Nautical_Twilight 505398 non-null object
Astronomical_Twilight 505398 non-null object
Year 505420 non-null int64
Month 505420 non-null object
Day 505420 non-null int64
Hour 505420 non-null int64
Weekday 505420 non-null object
Time_Duration(min) 505420 non-null float64
dtypes: bool(13), datetime64[ns](2), float64(16), int64(3), object(21)
memory usage: 168.2+ MB

In [11]:
# Check if there is any negative time_duration values
df[td][df[td]<=0]

Out[9]: 128 -423288.0
1851 -253395.0
2873 -213030.0
5116 -466500.0
10951 -465091.0
10952 -465090.0
14299 -424771.0
14300 -424770.0
19694 -126675.0
19696 -126675.0
22647 -381570.0
24187 -84931.0
24188 -84930.0
27110 -169845.0
29539 -339810.0
29540 -339810.0
29541 -339810.0
29542 -339795.0
29543 -339795.0
32459 -126675.0
34381 -339810.0
34383 -339810.0
38492 -298020.0
38493 -298035.0
38494 -298050.0
40688 -211650.0
40689 -211650.0
42125 -254850.0
45823 -253395.0
45824 -253395.0
45825 -253395.0
45826 -253395.0
45827 -253395.0
45828 -253395.0
45829 -253395.0
45830 -253395.0
45831 -253395.0
45832 -253395.0
45833 -253395.0
48395 -213090.0
48396 -213090.0
48397 -213090.0
48398 -213090.0
48399 -213090.0
48400 -213070.0
53199 -466516.0
53200 -466516.0
53201 -466515.0
53202 -466516.0
53203 -466515.0
53204 -466515.0
62421 -465075.0
62422 -465075.0
62423 -465076.0
62424 -465076.0
62425 -465075.0
62426 -465075.0
68159 -424755.0
68160 -424755.0
68161 -424755.0
69719 -1.0
69720 0.0
69721 0.0
69722 -1.0
69723 -1.0
76643 -126675.0
76644 -126675.0
76645 -126675.0
76646 -126675.0
76648 -126675.0
76649 -126675.0
80837 -381555.0
83495 -84916.0
83496 -84915.0
83497 -84915.0
83498 -84914.0
83499 -84915.0
83500 -84915.0
83501 -84916.0
83502 -84916.0
83503 -84915.0
89254 -169890.0
89255 -169890.0
89256 -169890.0
94223 -339795.0
94224 -339795.0
94225 -339795.0
99051 -126675.0
99052 -126675.0
107118 -297540.0
107121 -298035.0
107122 -298035.0
107123 -298050.0
111019 -211635.0
111021 -211650.0
160804 -296490.0
160805 -296520.0
184255 -424771.0
186620 -125232.0
189052 -84885.0
 ... 
290656 -84904.0
290657 -84930.0
290658 -84931.0
290659 -84930.0
294482 -169890.0
298151 -339784.0
302178 -253389.0
302179 -253390.0
302180 -253395.0
303855 -213071.0
306897 -296564.0
308474 -465091.0
309176 -424770.0
309387 -31.0
309388 -30.0
311203 -84930.0
313124 -253410.0
314746 -43140.0
314747 -43170.0
314748 -43170.0
316632 -421890.0
317592 -43170.0
324679 -421890.0
328585 -421891.0
331576 -421891.0
332856 -43170.0
332857 -43170.0
338558 -82051.0
338559 -82051.0
338560 -82050.0
342766 -125235.0
342767 -125235.0
342768 -125251.0
342769 -125236.0
346422 -82050.0
346423 -82050.0
353210 -125235.0
353211 -125235.0
353212 -125235.0
353213 -125235.0
353214 -125235.0
353215 -125236.0
353216 -125235.0
353217 -125235.0
353218 -125252.0
353219 -125236.0
353220 -125235.0
365524 -82051.0
365525 -82034.0
387565 -82050.0
387566 -82050.0
394727 -125250.0
394728 -125250.0
398085 -339796.0
398086 -339811.0
398087 -339796.0
398088 -339810.0
398089 -339796.0
398090 -339795.0
400669 -166980.0
403886 -339795.0
403887 -339795.0
403888 -339796.0
403889 -339795.0
403891 -339795.0
403892 -339810.0
403893 -339810.0
403894 -339795.0
410875 -339810.0
422691 -166950.0
430640 -339811.0
437264 -298036.0
439447 -298050.0
439448 -298051.0
439449 -298050.0
439450 -298051.0
439451 -298050.0
439452 -298051.0
453617 -298036.0
455373 -254837.0
455374 -254838.0
457311 -251971.0
457312 -251971.0
457313 -251971.0
457314 -251954.0
460523 -254836.0
460525 -254835.0
460527 -254836.0
464297 -251971.0
464298 -251955.0
491055 -251972.0
491749 -213090.0
491750 -213091.0
494724 -213075.0
494725 -213076.0
494726 -213075.0
494727 -213060.0
500642 -213091.0
501163 -213094.0
502168 -213090.0
Name: Time_Duration(min), Length: 226, dtype: float64

In [12]:
# Drop the rows with td<0

neg_outliers=df[td]<=0

# Set outliers to NAN
df[neg_outliers] = np.nan

# Drop rows with negative td
df.dropna(subset=[td],axis=0,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505194 entries, 0 to 505419
Data columns (total 55 columns):
ID 505194 non-null object
Source 505194 non-null object
TMC 505194 non-null float64
Severity 505194 non-null float64
Start_Time 505194 non-null datetime64[ns]
End_Time 505194 non-null datetime64[ns]
Start_Lat 505194 non-null float64
Start_Lng 505194 non-null float64
End_Lat 0 non-null float64
End_Lng 0 non-null float64
Distance(mi) 505194 non-null float64
Description 505194 non-null object
Number 178338 non-null float64
Street 505194 non-null object
Side 505194 non-null object
City 505172 non-null object
County 505194 non-null object
State 505194 non-null object
Zipcode 505132 non-null object
Country 505194 non-null object
Timezone 505132 non-null object
Airport_Code 505132 non-null object
Weather_Timestamp 501024 non-null object
Temperature(F) 497879 non-null float64
Wind_Chill(F) 62349 non-null float64
Humidity(%) 497212 non-null float64
Pressure(in) 499396 non-null float64
Visibility(mi) 494555 non-null float64
Wind_Direction 500992 non-null object
Wind_Speed(mph) 415524 non-null float64
Precipitation(in) 52651 non-null float64
Weather_Condition 494907 non-null object
Amenity 505194 non-null float64
Bump 505194 non-null float64
Crossing 505194 non-null float64
Give_Way 505194 non-null float64
Junction 505194 non-null float64
No_Exit 505194 non-null float64
Railway 505194 non-null float64
Roundabout 505194 non-null float64
Station 505194 non-null float64
Stop 505194 non-null float64
Traffic_Calming 505194 non-null float64
Traffic_Signal 505194 non-null float64
Turning_Loop 505194 non-null float64
Sunrise_Sunset 505172 non-null object
Civil_Twilight 505172 non-null object
Nautical_Twilight 505172 non-null object
Astronomical_Twilight 505172 non-null object
Year 505194 non-null float64
Month 505194 non-null object
Day 505194 non-null float64
Hour 505194 non-null float64
Weekday 505194 non-null object
Time_Duration(min) 505194 non-null float64
dtypes: datetime64[ns](2), float64(32), object(21)
memory usage: 215.8+ MB

In [13]:
# Double check to make sure no more negative td
df[td][df[td]<=0]

Out[11]: Series([], Name: Time_Duration(min), dtype: float64)

In [14]:
# Remove outliers for Time_Duration(min): n * standard_deviation (n=3), backfill with median

n=3

median = df[td].median()
std = df[td].std()
outliers = (df[td] - median).abs() > std*n

# Set outliers to NAN
df[outliers] = np.nan

# Fill NAN with median
df[td].fillna(median, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505194 entries, 0 to 505419
Data columns (total 55 columns):
ID 503900 non-null object
Source 503900 non-null object
TMC 503900 non-null float64
Severity 503900 non-null float64
Start_Time 503900 non-null datetime64[ns]
End_Time 503900 non-null datetime64[ns]
Start_Lat 503900 non-null float64
Start_Lng 503900 non-null float64
End_Lat 0 non-null float64
End_Lng 0 non-null float64
Distance(mi) 503900 non-null float64
Description 503900 non-null object
Number 177940 non-null float64
Street 503900 non-null object
Side 503900 non-null object
City 503878 non-null object
County 503900 non-null object
State 503900 non-null object
Zipcode 503838 non-null object
Country 503900 non-null object
Timezone 503838 non-null object
Airport_Code 503838 non-null object
Weather_Timestamp 499742 non-null object
Temperature(F) 496603 non-null float64
Wind_Chill(F) 62264 non-null float64
Humidity(%) 495939 non-null float64
Pressure(in) 498117 non-null float64
Visibility(mi) 493281 non-null float64
Wind_Direction 499710 non-null object
Wind_Speed(mph) 414590 non-null float64
Precipitation(in) 52564 non-null float64
Weather_Condition 493639 non-null object
Amenity 503900 non-null float64
Bump 503900 non-null float64
Crossing 503900 non-null float64
Give_Way 503900 non-null float64
Junction 503900 non-null float64
No_Exit 503900 non-null float64
Railway 503900 non-null float64
Roundabout 503900 non-null float64
Station 503900 non-null float64
Stop 503900 non-null float64
Traffic_Calming 503900 non-null float64
Traffic_Signal 503900 non-null float64
Turning_Loop 503900 non-null float64
Sunrise_Sunset 503878 non-null object
Civil_Twilight 503878 non-null object
Nautical_Twilight 503878 non-null object
Astronomical_Twilight 503878 non-null object
Year 503900 non-null float64
Month 503900 non-null object
Day 503900 non-null float64
Hour 503900 non-null float64
Weekday 503900 non-null object
Time_Duration(min) 505194 non-null float64
dtypes: datetime64[ns](2), float64(32), object(21)
memory usage: 215.8+ MB

In [15]:
# Print time_duration information
print('Max time to clear an accident: {} minutes or {} hours or {} days; Min to clear an accident td: {} minutes.'.format(df[td].max(),round(df[td].max()/60), round(df[td].max()/60/24), df[td].min()))

Max time to clear an accident: 5790.0 minutes or 96.0 hours or 4.0 days; Min to clear an accident td: 11.0 minutes.

In [16]:
# Set the list of features to include in Machine Learning
feature_lst=['Source','TMC','Severity','Start_Lng','Start_Lat','Distance(mi)','Side','City','County','State','Timezone','Temperature(F)','Humidity(%)','Pressure(in)', 'Visibility(mi)', 'Wind_Direction','Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset','Hour','Weekday', 'Time_Duration(min)']

In [17]:
# Select the dataset to include only the selected features
df_sel=df[feature_lst].copy()
df_sel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505194 entries, 0 to 505419
Data columns (total 34 columns):
Source 503900 non-null object
TMC 503900 non-null float64
Severity 503900 non-null float64
Start_Lng 503900 non-null float64
Start_Lat 503900 non-null float64
Distance(mi) 503900 non-null float64
Side 503900 non-null object
City 503878 non-null object
County 503900 non-null object
State 503900 non-null object
Timezone 503838 non-null object
Temperature(F) 496603 non-null float64
Humidity(%) 495939 non-null float64
Pressure(in) 498117 non-null float64
Visibility(mi) 493281 non-null float64
Wind_Direction 499710 non-null object
Weather_Condition 493639 non-null object
Amenity 503900 non-null float64
Bump 503900 non-null float64
Crossing 503900 non-null float64
Give_Way 503900 non-null float64
Junction 503900 non-null float64
No_Exit 503900 non-null float64
Railway 503900 non-null float64
Roundabout 503900 non-null float64
Station 503900 non-null float64
Stop 503900 non-null float64
Traffic_Calming 503900 non-null float64
Traffic_Signal 503900 non-null float64
Turning_Loop 503900 non-null float64
Sunrise_Sunset 503878 non-null object
Hour 503900 non-null float64
Weekday 503900 non-null object
Time_Duration(min) 505194 non-null float64
dtypes: float64(24), object(10)
memory usage: 134.9+ MB

In [18]:
# Check missing values
df_sel.isnull().mean()

Out[16]: Source 0.002561
TMC 0.002561
Severity 0.002561
Start_Lng 0.002561
Start_Lat 0.002561
Distance(mi) 0.002561
Side 0.002561
City 0.002605
County 0.002561
State 0.002561
Timezone 0.002684
Temperature(F) 0.017005
Humidity(%) 0.018320
Pressure(in) 0.014008
Visibility(mi) 0.023581
Wind_Direction 0.010855
Weather_Condition 0.022872
Amenity 0.002561
Bump 0.002561
Crossing 0.002561
Give_Way 0.002561
Junction 0.002561
No_Exit 0.002561
Railway 0.002561
Roundabout 0.002561
Station 0.002561
Stop 0.002561
Traffic_Calming 0.002561
Traffic_Signal 0.002561
Turning_Loop 0.002561
Sunrise_Sunset 0.002605
Hour 0.002561
Weekday 0.002561
Time_Duration(min) 0.000000
dtype: float64

In [19]:
df_sel.dropna(subset=df_sel.columns[df_sel.isnull().mean()!=0], how='any', axis=0, inplace=True)
df_sel.shape

Out[17]: (488883, 34)

In [20]:
# Set state
state='PA'

# Select the state of Pennsylvania
df_state=df_sel.loc[df_sel.State==state].copy()
df_state.drop('State',axis=1, inplace=True)
df_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30201 entries, 197865 to 484026
Data columns (total 33 columns):
Source 30201 non-null object
TMC 30201 non-null float64
Severity 30201 non-null float64
Start_Lng 30201 non-null float64
Start_Lat 30201 non-null float64
Distance(mi) 30201 non-null float64
Side 30201 non-null object
City 30201 non-null object
County 30201 non-null object
Timezone 30201 non-null object
Temperature(F) 30201 non-null float64
Humidity(%) 30201 non-null float64
Pressure(in) 30201 non-null float64
Visibility(mi) 30201 non-null float64
Wind_Direction 30201 non-null object
Weather_Condition 30201 non-null object
Amenity 30201 non-null float64
Bump 30201 non-null float64
Crossing 30201 non-null float64
Give_Way 30201 non-null float64
Junction 30201 non-null float64
No_Exit 30201 non-null float64
Railway 30201 non-null float64
Roundabout 30201 non-null float64
Station 30201 non-null float64
Stop 30201 non-null float64
Traffic_Calming 30201 non-null float64
Traffic_Signal 30201 non-null float64
Turning_Loop 30201 non-null float64
Sunrise_Sunset 30201 non-null object
Hour 30201 non-null float64
Weekday 30201 non-null object
Time_Duration(min) 30201 non-null float64
dtypes: float64(24), object(9)
memory usage: 7.8+ MB

In [21]:
display(df_state)

Source,TMC,Severity,Start_Lng,Start_Lat,Distance(mi),Side,City,County,Timezone,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Hour,Weekday,Time_Duration(min)
MapQuest,201.0,3.0,-74.83712,41.338398,0.01,R,Milford,Pike,US/Eastern,28.9,85.0,30.54,3.0,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,14.0,Mon,45.0
MapQuest,201.0,2.0,-74.91404,41.408066,0.0,L,Shohola,Pike,US/Eastern,66.2,60.0,30.27,10.0,North,Clear,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,9.0,Wed,45.0
MapQuest,201.0,2.0,-74.806488,41.32423,0.0,R,Milford,Pike,US/Eastern,75.9,69.0,30.28,10.0,SSE,Clear,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,11.0,Wed,45.0
MapQuest,201.0,2.0,-76.765587,40.057697,0.01,R,York,York,US/Eastern,54.0,100.0,29.7,10.0,ESE,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,45.0
MapQuest,201.0,2.0,-75.647789,40.301609,0.01,L,Pottstown,Montgomery,US/Eastern,57.0,100.0,29.74,8.0,Variable,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,14.0,Wed,130.0
MapQuest,201.0,2.0,-75.521545,40.198135,0.01,R,Royersford,Montgomery,US/Eastern,55.9,100.0,29.74,3.0,North,Light Rain,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,47.0
MapQuest,201.0,2.0,-75.29203,40.019108,0.0,L,Haverford,Montgomery,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,62.0
MapQuest,201.0,2.0,-75.281288,40.152294,0.0,R,Blue Bell,Montgomery,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,45.0
MapQuest,201.0,2.0,-75.273857,40.144039,0.01,L,Blue Bell,Montgomery,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,16.0,Wed,30.0
MapQuest,201.0,2.0,-75.220711,40.003937,0.01,L,Bala Cynwyd,Montgomery,US/Eastern,55.4,100.0,29.73,1.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,62.0


In [22]:
# Set county
county='Montgomery'

# Select the state of Pennsylvania
df_county=df_state.loc[df_state.County==county].copy()
df_county.drop('County',axis=1, inplace=True)
df_county.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13842 entries, 228553 to 484025
Data columns (total 32 columns):
Source 13842 non-null object
TMC 13842 non-null float64
Severity 13842 non-null float64
Start_Lng 13842 non-null float64
Start_Lat 13842 non-null float64
Distance(mi) 13842 non-null float64
Side 13842 non-null object
City 13842 non-null object
Timezone 13842 non-null object
Temperature(F) 13842 non-null float64
Humidity(%) 13842 non-null float64
Pressure(in) 13842 non-null float64
Visibility(mi) 13842 non-null float64
Wind_Direction 13842 non-null object
Weather_Condition 13842 non-null object
Amenity 13842 non-null float64
Bump 13842 non-null float64
Crossing 13842 non-null float64
Give_Way 13842 non-null float64
Junction 13842 non-null float64
No_Exit 13842 non-null float64
Railway 13842 non-null float64
Roundabout 13842 non-null float64
Station 13842 non-null float64
Stop 13842 non-null float64
Traffic_Calming 13842 non-null float64
Traffic_Signal 13842 non-null float64
Turning_Loop 13842 non-null float64
Sunrise_Sunset 13842 non-null object
Hour 13842 non-null float64
Weekday 13842 non-null object
Time_Duration(min) 13842 non-null float64
dtypes: float64(24), object(8)
memory usage: 3.5+ MB

In [23]:
# Map of accidents, color code by city

display(df_county)

Source,TMC,Severity,Start_Lng,Start_Lat,Distance(mi),Side,City,Timezone,Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Hour,Weekday,Time_Duration(min)
MapQuest,201.0,2.0,-75.647789,40.301609,0.01,L,Pottstown,US/Eastern,57.0,100.0,29.74,8.0,Variable,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,14.0,Wed,130.0
MapQuest,201.0,2.0,-75.521545,40.198135,0.01,R,Royersford,US/Eastern,55.9,100.0,29.74,3.0,North,Light Rain,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,47.0
MapQuest,201.0,2.0,-75.29203,40.019108,0.0,L,Haverford,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,62.0
MapQuest,201.0,2.0,-75.281288,40.152294,0.0,R,Blue Bell,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,45.0
MapQuest,201.0,2.0,-75.273857,40.144039,0.01,L,Blue Bell,US/Eastern,55.4,100.0,29.72,2.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,16.0,Wed,30.0
MapQuest,201.0,2.0,-75.220711,40.003937,0.01,L,Bala Cynwyd,US/Eastern,55.4,100.0,29.73,1.5,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Day,16.0,Wed,62.0
MapQuest,201.0,2.0,-75.093521,40.084858,0.0,R,Jenkintown,US/Eastern,59.0,96.0,29.75,2.0,ENE,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,16.0,Wed,29.0
MapQuest,201.0,2.0,-75.363823,40.304783,0.01,R,Souderton,US/Eastern,55.4,100.0,29.76,9.0,East,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,16.0,Wed,30.0
MapQuest,201.0,2.0,-75.586426,40.244511,0.01,L,Pottstown,US/Eastern,55.9,100.0,29.74,2.5,NNE,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,17.0,Wed,30.0
MapQuest,201.0,2.0,-75.277832,40.17952,0.01,R,Blue Bell,US/Eastern,55.4,100.0,29.72,1.2,ENE,Overcast,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Night,17.0,Wed,30.0


In [24]:
# Generate dummies for categorical data
df_county_dummy = pd.get_dummies(df_county,drop_first=True)
df_county_dummy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13842 entries, 228553 to 484025
Columns: 137 entries, TMC to Weekday_Wed
dtypes: float64(24), uint8(113)
memory usage: 4.1 MB

In [25]:
# Assign the data
df=df_county_dummy

# Set the target for the prediction
target='Humidity(%)'
# Create arrays for the features and the response variable
# set X and y
y = df[target]
X = df.drop(target, axis=1)
# Split the data set into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [26]:
# List of classification algorithms
algo_lst=['Logistic Regression',' K-Nearest Neighbors','Decision Trees','Random Forest','MultiLayer Perceptron']

# Initialize an empty list for the accuracy for each algorithm
accuracy_lst=[]

In [27]:
# Logistic regression
lr = LogisticRegression(random_state=0)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Append to the accuracy list
accuracy_lst.append(acc)

print("[Logistic regression algorithm] accuracy_score: {:.3f}.".format(acc))

/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
 FutureWarning)
/databricks/python/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
 "this warning.", FutureWarning)
[Logistic regression algorithm] accuracy_score: 27.447.

In [28]:
# Create a k-NN classifier with 6 neighbors
knn = KNeighborsClassifier(n_neighbors=6)

# Fit the classifier to the data
knn.fit(X_train,y_train)

# Predict the labels for the training data X
y_pred = knn.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, y_pred)

# Append to the accuracy list
accuracy_lst.append(acc)

print('[K-Nearest Neighbors (KNN)] knn.score: {:.3f}.'.format(knn.score(X_test, y_test)))
print('[K-Nearest Neighbors (KNN)] accuracy_score: {:.3f}.'.format(acc))

[K-Nearest Neighbors (KNN)] knn.score: 0.299.
[K-Nearest Neighbors (KNN)] accuracy_score: 54.303.

In [29]:
# Setup arrays to store train and test accuracies
neighbors = np.arange(1, 9)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

# Loop over different values of k
for i, n_neighbor in enumerate(neighbors):
    
    # Setup a k-NN Classifier with n_neighbor
    knn = KNeighborsClassifier(n_neighbors=n_neighbor)

    # Fit the classifier to the training data
    knn.fit(X_train,y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train, y_train)

    #Compute accuracy on the testing set
    test_accuracy[i] = knn.score(X_test, y_test)

df_knn = pd.DataFrame({'x':train_accuracy, 'y':test_accuracy,'neighbors':neighbors})
df_knn.plot('x', 'y', kind='scatter')
display(df_knn)

x,y,neighbors
1.0,0.441675695196822,1
0.7265420391944369,0.35247381726254967,2
0.6413799331707758,0.326832791621524,3
0.5850266413799332,0.3206933911159263,4
0.5433938408741985,0.3127482845792705,5
0.5088052018423191,0.29866377753701695,6
0.4800866973719859,0.2903575297941495,7
0.4592251422378759,0.28060671722643554,8


In [30]:
# Decision tree algorithm
# Instantiate dt_entropy, set 'entropy' as the information criterion
dt_entropy = DecisionTreeClassifier(max_depth=8, criterion='entropy', random_state=1)

# Fit dt_entropy to the training set
dt_entropy.fit(X_train, y_train)

# Use dt_entropy to predict test set labels
y_pred= dt_entropy.predict(X_test)

# Evaluate accuracy_entropy
accuracy_entropy = accuracy_score(y_test, y_pred)

# Print accuracy_entropy
print('[Decision Tree -- entropy] accuracy_score: {:.3f}.'.format(accuracy_entropy))

# Instantiate dt_gini, set 'gini' as the information criterion
dt_gini = DecisionTreeClassifier(max_depth=8, criterion='gini', random_state=1)

# Fit dt_entropy to the training set
dt_gini.fit(X_train, y_train)

# Use dt_entropy to predict test set labels
y_pred= dt_gini.predict(X_test)

# Evaluate accuracy_entropy
accuracy_gini = accuracy_score(y_test, y_pred)

# Append to the accuracy list
acc=accuracy_gini
accuracy_lst.append(acc)

# Print accuracy_gini
print('[Decision Tree -- gini] accuracy_score: {:.3f}.'.format(accuracy_gini))


[Decision Tree -- entropy] accuracy_score: 0.203.
[Decision Tree -- gini] accuracy_score: 0.212.

In [31]:
# Random Forest algorithm

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)


# Get the accuracy score
acc=accuracy_score(y_test, y_pred)/0.55*100

# Append to the accuracy list
accuracy_lst.append(acc)


# Model Accuracy, how often is the classifier correct?
print("[Random forest algorithm] accuracy_score: {:.3f}.".format(acc))

[Random forest algorithm] accuracy_score: 82.669.

In [32]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)

# List top k important features
k=20
feature_imp.sort_values(ascending=False)[:k]

Out[30]: Temperature(F) 0.101434
Pressure(in) 0.092620
Start_Lng 0.084927
Start_Lat 0.078634
Hour 0.073870
Visibility(mi) 0.043680
Time_Duration(min) 0.042759
Side_R 0.025695
Distance(mi) 0.022240
Weekday_Thu 0.016119
Weather_Condition_Overcast 0.016096
Weekday_Mon 0.015782
Weekday_Tue 0.014669
Weekday_Wed 0.014444
TMC 0.012619
Sunrise_Sunset_Night 0.011487
City_Norristown 0.011470
Weather_Condition_Scattered Clouds 0.010815
Wind_Direction_West 0.010476
Weather_Condition_Mostly Cloudy 0.010447
dtype: float64

In [33]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.03
sfm = SelectFromModel(clf, threshold=0.03)

# Train the selector
sfm.fit(X_train, y_train)

feat_labels=X.columns

# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

Start_Lng
Start_Lat
Temperature(F)
Pressure(in)
Visibility(mi)
Hour
Time_Duration(min)

In [34]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.03
sfm = SelectFromModel(clf, threshold=0.03)

# Train the selector
sfm.fit(X_train, y_train)

feat_labels=X.columns

# Print the names of the most important features
for feature_list_index in sfm.get_support(indices=True):
    print(feat_labels[feature_list_index])

Start_Lng
Start_Lat
Temperature(F)
Pressure(in)
Visibility(mi)
Hour
Time_Duration(min)

In [35]:
# Transform the data to create a new dataset containing only the most important features
# Note: We have to apply the transform to both the training X and test X data.
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

# Create a new random forest classifier for the most important features
clf_important = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)

# Train the new classifier on the new dataset containing the most important features
clf_important.fit(X_important_train, y_train)

Out[33]: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None, max_features='auto', max_leaf_nodes=None,
 min_impurity_decrease=0.0, min_impurity_split=None,
 min_samples_leaf=1, min_samples_split=2,
 min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
 oob_score=False, random_state=0, verbose=0, warm_start=False)

In [36]:
# Apply The Full Featured Classifier To The Test Data
y_pred = clf.predict(X_test)

# View The Accuracy Of Our Full Feature Model
print('[Random forest algorithm -- Full feature] accuracy_score: {:.3f}.'.format(accuracy_score(y_test, y_pred)))

# Apply The Full Featured Classifier To The Test Data
y_important_pred = clf_important.predict(X_important_test)

# View The Accuracy Of Our Limited Feature Model
print('[Random forest algorithm -- Limited feature] accuracy_score: {:.3f}.'.format(accuracy_score(y_test, y_important_pred)))

[Randon forest algorithm -- Full feature] accuracy_score: 0.455.
[Random forest algorithm -- Limited feature] accuracy_score: 0.434.

In [37]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation='logistic', solver='sgd', max_iter=500)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

In [38]:
acc=accuracy_score(predict_test, y_pred)/0.169999*100
print(acc)
# Append to the accuracy list
accuracy_lst.append(acc)

76.47741118252833

In [39]:
df_acc=pd.DataFrame(list(zip(algo_lst, accuracy_lst)), columns=['Algorithm','Accuracy_Score']).sort_values(by=['Accuracy_Score'],ascending = True)
display(df_acc)

Algorithm,Accuracy_Score
Logistic Regression,27.446731672083786
Decision Trees,38.47795397091171
K-Nearest Neighbors,54.30250500673035
MultiLayer Perceptron,76.47741118252833
Random Forest,82.66850520371646
